# nn.Module

In [ ]:
import torch
import torch.nn as nn  
import torch.nn.functional as F 

## 1. `nn.Module`

```python

CLASS torch.nn.Module  # 为所有神经网络提供基本骨架, 搭建的 Model都必须继承该类
```

模板:

```python

class MyNet(nn.Module):   #搭建的神经网络 MyNet 继承了 Module类
    def __init__(self):   
        super(Model, self).__init__()   #必须要这一步，调用父类的初始化函数
        self.conv1 = nn.Conv2d(3, 3, 5)
        self.conv2 = nn.Conv2d(3, 3, 5)
 
    def forward(self, x):   #前向传播（为输入和输出中间的处理过程）, x为输入
        x = F.relu(self.conv1(x))   
        return F.relu(self.conv2(x))

```

## 2. 卷积,池化,非线性激活,全连接

### 2.1.  2D卷积,`nn.Conv2d`和`F.conv2d`

- `nn.Conv2d`(类式接口)
- `F.conv2d`(函数式接口): 函数式的更加low-level一些，如果不需要做特别复杂的配置只要用**类式接口**就够了
- 可以这样理解：`nn.Conv2d`是[2D卷积层]，而`F.conv2d`是[2D卷积操作]

---------------------------------------------------------------  <p>

**`nn.Conv2d()`的参数:** <p>
`nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True,padding_mode='zeros')`
- in_channels: 输入图像的通道数
- out_channels: 卷积生成的通道数
- kernel_size:  指卷积核的长宽,如为3则卷积核是3乘3的

**`F.conv2d()`的参数:** <p>
`nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) → Tensor`

- **input:** input tensor of shape (batchszie , in_channels , Height , Width)
- **weight:** weight tensor of shape (out_channels , in_channels/groups , H , W)

In [ ]:
"""-----------nn.Conv2d()------------------"""

# 输入图像的通道数 =1 , 卷积核的种类数=3
conv_layer = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=0)

# 样本数=1,通道数=1,图像的shape是5乘5的
input = torch.rand(1, 1, 5, 5)

conv_out = conv_layer.forward(input)
conv_out1 = conv_layer(input)   # 更常用

# 得到的还是1张图片,因为用了3种kernel所以输出的通道数变成3了
print(conv_layer.weight)
print("---------------------------------------")
print(conv_layer.bias)
print("---------------------------------------")
print(conv_out.shape)  # torch.Size([1, 3, 3, 3])
print(conv_out1.shape)
print("---------------------------------------")


"""-----------F.conv2d()------------------"""

kernel = torch.tensor([[1,2,1],
                       [0,1,0],
                       [2,1,0]],dtype = torch.float32 )

kernel = torch.reshape(kernel,(1,1,3,3))

output = F.conv2d(input,kernel)
print("The output of nn.conv2d() is:")
print(f'{output}, with shape {output.shape}')
print("---------------------------------------")

tensor([[[[0.3487, 0.5030, 0.6218, 0.6449, 0.0802],
          [0.1641, 0.5387, 0.9858, 0.6082, 0.9344],
          [0.9847, 0.7799, 0.6181, 0.5815, 0.5670],
          [0.9748, 0.2992, 0.3843, 0.8859, 0.4536],
          [0.6485, 0.6504, 0.2218, 0.2132, 0.3070]]]])
Parameter containing:
tensor([[[[-0.2211, -0.0562, -0.2652],
          [-0.1037,  0.2803,  0.2123],
          [-0.2866,  0.0138,  0.0752]]],


        [[[ 0.0688,  0.0723, -0.2975],
          [ 0.0280, -0.0901,  0.0888],
          [-0.0455,  0.2577,  0.1442]]],


        [[[ 0.1979,  0.0342, -0.2526],
          [ 0.0367, -0.1139, -0.3222],
          [ 0.0890, -0.3165, -0.0681]]]], requires_grad=True)
---------------------------------------
Parameter containing:
tensor([ 0.0866, -0.2517, -0.1145], requires_grad=True)
---------------------------------------
torch.Size([1, 3, 3, 3])
torch.Size([1, 3, 3, 3])
---------------------------------------
The output of nn.conv2d() is:
tensor([[[[5.2646, 5.5551, 4.4175],
          [5.2560, 

### 2.2. 池化层

- **池化:** 池化层是卷积神经网络中用于减小参数数量和计算复杂度的常用技术。池化层通过在输入特征图上滑动一个固定大小的窗口，并计算窗口内元素的最大值、最小值或平均值，从而生成一个新的特征图。

- **Pool** Vs **Conv**

    - **池化与卷积的共同点:** 池化操作也是原图像矩阵与一个固定形状的窗口(kernel))进行计算，并输出特征图的一种计算方式;

    - **池化与卷积的不同点:** 卷积操作的卷积核是有数据(权重)的，而池化直接计算池化窗口内的原始数据，这个计算过程可以是选择最大值、选择最小值或计算平均值，分别对应：最大池化、最小池化和平均池化。在实际使用中最大池化是应用最广泛的池化方法。


- **池化种类**
    - MaxPool：最大池化(下采样): 提取出特定窗口的最大数据. 
    - AvgPool：平均池化
    - AdaptiveMaxPool2d：自适应最大池化


In [32]:
maxpool_layer = nn.MaxPool2d(kernel_size=3, stride=1, padding=0)
maxpool_output = maxpool_layer(input)   # input-shape: (1,1,5,5)
print(maxpool_output)

tensor([[[[0.9171, 0.9558, 0.9952],
          [0.9171, 0.9558, 0.9558],
          [0.8920, 0.9558, 0.9558]]]])


### 2.3. 非线性激活函数

- `Sigmod()`: **$$\operatorname{sigmoid}(x) = \frac{1}{1 + \exp(-x)}.$$**
              $$\frac{d}{dx} \operatorname{sigmoid}(x) = \frac{\exp(-x)}{(1 + \exp(-x))^2} = \operatorname{sigmoid}(x)\left(1-\operatorname{sigmoid}(x)\right).$$
其优点主要是连续，平滑便于求导. 但是其的缺点也很致命:梯度消失问题,当x>2或x<2时Sigmod输出趋于平滑，导致梯度减小,权重和偏置更新过慢导致网络不更新; 非零均值特性:会使训练震荡达不到最优解，使收敛变慢; 导数计算复杂，影响速度

- `Tanh()`: Tanh主要解决了Sigmod非零均值特性的问题，但是其还是存在计算复杂和梯度消失的问题.
**$$\operatorname{tanh}(x) = \frac{1 - \exp(-2x)}{1 + \exp(-2x)}.$$**
$$\frac{d}{dx} \operatorname{tanh}(x) = 1 - \operatorname{tanh}^2(x).$$

- `ReLU()`:  **$$\operatorname{ReLU}(x) = \max(x, 0).$$**
主要优点有:大于0时，其导数恒为1，不会存在梯度消失的问题; 计算速度非常快，只需要判断 x 是大于0还是小于0; 收敛速度远远快于前面的 Sigmoid 和 Tanh函数。
但是ReLu也是有着缺陷的: 非零均值特性; x<0时，输出恒为0, 会使某些神经元永远不会被激活，进而导致参数永远不会更新

- `pReLU`: **$$\operatorname{pReLU}(x) = \max(0, x) + \alpha \min(0, x).$$**
- `Leaky ReLU`: **$$\operatorname{Leaky ReLU}(x) = \max(0.1x, x).$$**

- `Softmax()`: **$$\operatorname{softmax}(x)_i = \frac{\exp(x_i)}{\sum_{j} \exp(x_j)}.$$**

In [14]:
a = -5 # lowerlimit
b = 5 # upperlimit
input = a + (b - a) * torch.rand(1, 1, 5, 5)

relu = nn.ReLU(inplace=False) # 当 inplace=False 时，ReLU 会创建一个新的输出 Tensor。输入 Tensor 的值不会被修改
relu_output = relu(input)
print(relu_output) # 小于0的都被截断为0
print("---------------------------------------")
sigmod = nn.Sigmoid()
sig_output = sigmod(input)
print(sig_output)
print("---------------------------------------")

softmax = nn.Softmax(dim=3) # dim 参数指定了 Softmax 操作沿着哪个维度进行
softmax_output = softmax(input)
print(softmax_output)

tensor([[[[0.1294, 0.4077, 0.0000, 0.0000, 0.0000],
          [1.2515, 0.0000, 4.2310, 4.4363, 0.0000],
          [0.0000, 2.1058, 0.0000, 0.0000, 0.0000],
          [1.1504, 0.0000, 0.0000, 0.6268, 0.0000],
          [0.0000, 3.3425, 4.4721, 2.8028, 1.0364]]]])
---------------------------------------
tensor([[[[0.5323, 0.6005, 0.0428, 0.1636, 0.2994],
          [0.7776, 0.0161, 0.9857, 0.9883, 0.0817],
          [0.0351, 0.8915, 0.1776, 0.0550, 0.0082],
          [0.7596, 0.4569, 0.0108, 0.6518, 0.0483],
          [0.1298, 0.9659, 0.9887, 0.9428, 0.7382]]]])
---------------------------------------
tensor([[[[3.4395e-01, 4.5432e-01, 1.3499e-02, 5.9111e-02, 1.2912e-01],
          [2.2286e-02, 1.0406e-04, 4.3856e-01, 5.3849e-01, 5.6688e-04],
          [4.2658e-03, 9.6264e-01, 2.5301e-02, 6.8265e-03, 9.6661e-04],
          [5.3241e-01, 1.4180e-01, 1.8449e-03, 3.1540e-01, 8.5454e-03],
          [1.1026e-03, 2.0911e-01, 6.4706e-01, 1.2189e-01, 2.0837e-02]]]])


### 2.4. 线性连接层

`torch.nn.Linear(in_features, out_features, bias=True # 是否包含偏置)`

- **in_features** = i :  输入的神经元个数
- **out_features** = o : 输出神经元个数
- **bias**: 是否包含偏置

**Linear** 其实就是对输入 $X_{n \times i}$ **执行了一个线性变换**，即：

$$
\begin{aligned}
    Y_{n \times o} & = X_{n \times i} W_{i \times o} + b
\end{aligned}
$$

其中 $W$ 是模型想要学习的参数，$W$ 的维度为 $W_{i \times o}$，$b$ 是 $o$ 维的向量偏置，$n$ 为输入向量的行数（例如，你想一次输入 10 个样本，即 `batch_size` 为 10，n=10），$i$ 为输入神经元的个数（例如你的样本特征为 5，则 $i=5$），$o$ 为输出神经元的个数。



In [15]:
linear_layer = nn.Linear(2, 1) # 输入特征数为2，输出特征数为1

linear_input = torch.Tensor([1, 2]) # 给一个样本，该样本有2个特征（这两个特征的值分别为1和2）
linear_output = linear_layer(linear_input)
print(linear_output)
print("____________________________________")
# 查看模型参数
# 模型有3个参数，分别为两个权重和一个bias
for linear_param in linear_layer.parameters():
    print(linear_param)



tensor([-1.9150], grad_fn=<AddBackward0>)
____________________________________
Parameter containing:
tensor([[-0.2738, -0.6235]], requires_grad=True)
Parameter containing:
tensor([-0.3942], requires_grad=True)


In [16]:
# X每一行为一个特征

X = torch.Tensor([
    [0.1,0.2,0.3,0.3,0.3],
    [0.4,0.5,0.6,0.6,0.6],
    [0.7,0.8,0.9,0.9,0.9],
])

X = torch.reshape(X,(1,1,3,5))


linear_layer_2 = nn.Linear(in_features=5, out_features=2, bias=True)

print(linear_layer_2(X))

tensor([[[[-0.4976, -0.0667],
          [-0.6202,  0.0359],
          [-0.7428,  0.1385]]]], grad_fn=<AddBackward0>)


### 1.5. `nn.Flatten()`

`torch.nn.Flatten(start_dim=1, end_dim=- 1)`

- 作用：将连续的维度范围展平为张量。 经常在`nn.Sequential()`中出现，一般写在某个神经网络模型之后，用于对神经网络模型的输出进行处理，得到`tensor`类型的数据。
- `start_dim`和`end_dim`，分别表示开始的维度和终止的维度，默认值分别是1和-1，其中1表示第一维度，-1表示最后的维度。结合起来看意思就是从第一维度到最后一个维度全部给展平为张量。（注意：数据的维度是从0开始的，也就是存在第0维度，第一维度并不是真正意义上的第一个）

In [ ]:
input = torch.randn(32, 2, 5, 5)
# With default parameters
f = nn.Flatten()
output = f(input)
output.size()
#torch.Size([32, 50])

torch.Size([32, 50])

## 3. `nn.Sequential()`

`nn.Sequential()` 是一个序列容器，用于搭建神经网络的模块被按照被传入器构造的顺序添加到容器中。除此之外，一个包含神经网络模块的`OrderedDict`也可以被传入`nn.Sequential()`容器中。利用`nn.Sequential()`搭建好模型架构，模型前向传播时调用`forward()`方法，模型接收的输入首先被传入`nn.Sequential()`包含的第一个网络模块中。然后，第一个网络模块的输出传入第二个网络模块作为输入，按照顺序依次计算并传播，直到`nn.Sequential()`里的最后一个模块输出结果。<p>
-------------------------------------------------------------------------------------------------------- <p>
与一层一层的单独调用模块组成序列相比，`nn.Sequential()` 可以允许将整个容器视为单个模块（即相当于把多个模块封装成一个模块），`forward()`方法接收输入之后，`nn.Sequential()`按照内部模块的顺序自动依次计算并输出结果。这就意味着我们可以利用`nn.Sequential() `自定义自己的网络层。

```python
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(in_channel, in_channel, kernel_size),
                                    nn.BatchNorm2d(in_channel),
                                    nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(in_channel, in_channel),
                                    nn.BatchNorm2d(in_channel),
                                    nn.ReLU())
        self.layer3 = nn.Sequential(nn.Conv2d(in_channel, out_channel, kernel_size),
                                    nn.BatchNorm2d(out_channel),
                                    nn.ReLU())
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        return x
```

## 4. 批标准化层

ref: [Batch Normalization（BN）超详细解析](https://blog.csdn.net/weixin_44023658/article/details/105844861)

<p>

对输入采用 `Batch Normalization`, 可以加快神经网络的训练速度 (在原paper中, BN被建议插入在 **(每个)** ReLU激活层前面).

`CLASS torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)`

- `num_features`：输入数据的通道数, C from an expected input of size (N,C,H,W)
- `eps`：分母中添加的一个值，防止分母为0
- `momentum`：用于计算运行时的均值和方差的动量
- `affine`：当设为True时，会给该层添加可学习的缩放和平移参数


In [25]:
# With Learnable Parameters
m = nn.BatchNorm2d(3)
# Without Learnable Parameters
m = nn.BatchNorm2d(3, affine=False)
input = torch.randn(1, 3, 2, 2)
print(input)
print("_____________________________________")
output = m(input)
print(output)

tensor([[[[ 0.2246,  0.1058],
          [-1.0115,  0.1297]],

         [[ 0.3059, -0.7004],
          [ 1.4057, -0.6412]],

         [[ 0.2998, -0.7642],
          [-0.4625,  0.7384]]]])
_____________________________________
tensor([[[[ 0.7158,  0.4811],
          [-1.7253,  0.5284]],

         [[ 0.2490, -0.9253],
          [ 1.5325, -0.8562]],

         [[ 0.5814, -1.2018],
          [-0.6961,  1.3165]]]])


## 5. Dropout Layer

- 在训练过程中, 随机把一些 input(输入的tensor数据类型) 中的一些元素变为0, 变为0的概率为p.


`torch.nn.Dropout(p=0.5, inplace=False)`

- `p`：每个元素被置零的概率，默认值为0.5
- `inplace`：如果设置为True，将会原地操作，即不会拷贝输入数据，会节省内存，但会破坏输入数据，因此默认为False

```python
dropout = nn.Dropout(p=0.5)
input = torch.randn(2, 3, 5, 5)
output = dropout(input)
print(output)
```

## 6. Recurrent Layers & Transformer Layers

See [PyTorch Doc](https://pytorch.org/docs/stable/nn.html#recurrent-layers) 

## 7. `nn Loss Functions`

- `nn.L1Loss`
- `nn.MSELoss`


## Sequential 的使用

```python
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
 
# Using Sequential with OrderedDict. This is functionally the
# same as the above code
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
```

注意：<p>

假设输入 x 的尺寸是 (batch_size, 1, 32, 32)。<p>
经过第一个卷积层 self.conv1(x)，输出特征图的尺寸为 (batch_size, 16, 28, 28)。<p>
然后应用 ReLU 激活函数 F.relu(self.conv1(x))，不会改变特征图的尺寸，仍然是 (batch_size, 16, 28, 28)。<p>
最后执行最大池化操作 F.max_pool2d(F.relu(self.conv1(x)), (2, 2))。<p>
- 池化窗口大小为 (2, 2)，也就是高度和宽度都是 2。
- 池化操作会将特征图的高度和宽度各缩小一半。

因此，经过最大池化后，输出特征图的尺寸为 (batch_size, 16, 14, 14)。<p>
综上所述，输入 x 的尺寸为 (batch_size, 1, 32, 32)，经过第一个卷积层和最大池化层后，输出特征图的尺寸变为 (batch_size, 16, 14, 14)。

In [5]:
# 输入是一个 32 x 32 的矩阵（图像）

class Net(nn.Module):
# 我们定义了一个名为 Net 的类,它继承自 nn.Module。这个类定义了一个简单的卷积神经网络(CNN)模型
    
    def __init__(self):
        # 在 __init__ 方法中,我们定义了该模型的各个层.
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)  # 第一层卷积：1 input image channel, 6 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        # self.fc1、self.fc2 和 self.fc3 是三个全连接层,分别将 16*5*5 维的输入映射到 120 维、120 维到 84 维,最后到 10 维(对应 10 个类别)。
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # 计算输入张量 x 展平后的特征数量
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def forward(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # Max pooling over a (2, 2) window
        
        # If the pooling size is a square you can only specify a single number
        # 如果池化窗口的高度和宽度是相同的,那么可以只指定一个数字,PyTorch 会自动将其扩展为 (2, 2) 这样的二元元组。
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # 我们将二维的特征图 x 展平成一维向量
        
        # 接下来,我们将展平后的特征向量传递到三个全连接层 self.fc1, self.fc2 和 self.fc3
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


在模型中必须要定义 ``forward`` 函数，``backward``
函数（用来计算梯度）会被``autograd``自动创建。
可以在 ``forward`` 函数中使用任何针对 Tensor 的操作。

 ``net.parameters()``返回可被学习的参数（权重）列表和值



In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


测试随机输入32×32。
注：这个网络（LeNet）期望的输入大小是32×32，如果使用MNIST数据集来训练这个网络，请把图片大小重新调整到32×32。



In [8]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0055, -0.0183,  0.0429, -0.0562,  0.0609, -0.0518,  0.0226,  0.0052,
          0.0502,  0.0509]], grad_fn=<AddmmBackward0>)


将所有参数的梯度缓存清零，然后进行随机梯度的的反向传播：



In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

``torch.nn`` 只支持小批量输入。整个 ``torch.nn``包都只支持小批量样本，而不支持单个样本。

例如，``nn.Conv2d`` 接受一个4维的张量，每一维分别是``sSamples * nChannels * Height * Width（样本数*通道数*高*宽）``。
<p>

如果你有单个样本，只需使用 ``input.unsqueeze(0)`` 来添加其它的维数<p>

在继续之前，我们回顾一下到目前为止用到的类。

**回顾:**
  -  ``torch.Tensor``：一个自动调用 ``backward()``实现支持自动梯度计算的 *多维数组* ，并且保存关于这个向量的*梯度* .
  -  ``nn.Module``：神经网络模块。封装参数、移动到GPU上运行、导出、加载等。
  -  ``nn.Parameter``：一种变量，当把它赋值给一个``Module``时，被 *自动* 地注册为一个参数。
  -  ``autograd.Function``：实现一个自动求导操作的前向和反向定义，每个变量操作至少创建一个函数节点，每一个``Tensor``的操作都回创建一个接到创建``Tensor``和 *编码其历史* 的函数的``Function``节点。

**重点如下：**
  -  定义一个网络
  -  处理输入，调用backword

**还剩：**
  -  计算损失
  -  更新网络权重

损失函数
-------------
一个损失函数接受一对 (output, target) 作为输入，计算一个值来估计网络的输出和目标值相差多少。

***译者注：output为网络的输出，target为实际值***

nn包中有很多不同的[损失函数](https://pytorch.org/docs/nn.html#loss-functions)。
``nn.MSELoss``是一个比较简单的损失函数，它计算输出和目标间的**均方误差**，
例如：



In [15]:
output = net(input)
target = torch.randn(10)  # 随机值作为样例
target = target.view(1, -1)  # 使target和output的shape相同
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.6377, grad_fn=<MseLossBackward0>)


现在，如果在反向过程中跟随``loss`` ， 使用它的
``.grad_fn`` 属性，将看到如下所示的计算图。

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

所以，当我们调用 ``loss.backward()``时,整张计算图都会
根据loss进行微分，而且图中所有设置为``requires_grad=True``的张量
将会拥有一个随着梯度累积的``.grad`` 张量。

为了说明，让我们向后退几步:



In [19]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
--------
调用loss.backward()获得反向传播的误差。

但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用loss.backward()，并查看conv1层的偏差（bias）项在反向传播前后的梯度。




In [10]:
net.zero_grad()     # 清除梯度

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0051,  0.0042,  0.0026,  0.0152, -0.0040, -0.0036])


如何使用损失函数

**稍后阅读：**

  `nn`包，包含了各种用来构成深度神经网络构建块的模块和损失函数，完整的文档请查看[here](https://pytorch.org/docs/nn)。

**剩下的最后一件事:**

  - 新网络的权重

更新权重
------------------
在实践中最简单的权重更新规则是随机梯度下降（SGD）： ``weight = weight - learning_rate * gradient``

我们可以使用简单的Python代码实现这个规则：

```python

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```
但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包``torch.optim``实现了所有的这些规则。
使用它们非常简单：


In [20]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. 注意::
    
观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。这是因为梯度是按Backprop部分中的说明累积的。



### Reference

- [[PyTorch学习笔记]17：2D卷积,nn.Conv2d和F.conv2d](https://blog.csdn.net/SHU15121856/article/details/88956545)
- [pytorch中nn.Sequential详解](https://blog.csdn.net/lsb2002/article/details/135096093)
- [Pytorch nn.Linear()的基本用法与原理详解及全连接层简介](https://blog.csdn.net/qq_44722189/article/details/135035351)